In [ ]:
!rm -r ./*
!git clone https://github.com/Mamiglia/BNN_Human_motion
!mv BNN_Human_motion/* .
!pip install bayesian-torch lightning

Cloning into 'BNN_Human_motion'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 368 (delta 55), reused 92 (delta 38), pack-reused 245
Receiving objects: 100% (368/368), 171.34 MiB | 18.41 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Updating files: 100% (249/249), done.


In [1]:
from funcs.dataloader import *
from funcs.autoformer.Autoformer import Model as autoformer
from funcs.loss import mpjpe_error
from funcs.autoformer.Light_Autoformer import LitAutoformer
import torch.optim as optim
import torch.autograd
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import lightning as L
# from lightning.pytorch.loggers import TensorBoardLogger

In [2]:
data, vald = load_dataset()

Loading Train Dataset...
Loading Validation Dataset...
>>> Training dataset length: 180077
>>> Validation dataset length: 28110


In [3]:
class Configs:
    def __init__(self, seq_len, pred_len, output_attention=False, moving_avg=3):
        self.seq_len = seq_len
        self.label_len = seq_len//2
        self.pred_len = pred_len
        self.output_attention = output_attention
        self.moving_avg = moving_avg
        self.enc_in=66
        self.dec_in=66
        self.d_model=66
        self.embed='fixed'
        self.freq='h'
        self.dropout=0.1
        self.e_layers=2
        self.d_layers=1
        self.c_out=66
        self.factor=1
        self.n_heads=1
        self.d_ff=64
        self.activation="relu"

In [4]:
device = "cuda"
configs = Configs(seq_len=10, pred_len=25)
autof = autoformer(configs).to(device)

In [5]:
# Arguments to setup the optimizer
lr=1e-03 # learning rate
use_scheduler=True # use MultiStepLR scheduler
milestones=[10,30]   # the epochs after which the learning rate is adjusted by gamma
gamma=0.1 #gamma correction to the learning rate, after reaching the milestone epochs
weight_decay=1e-07 # weight decay (L2 penalty)
optimizer=optim.Adam(autof.parameters(),lr=lr,weight_decay=weight_decay)

if use_scheduler:
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

clip_grad=None # select max norm to clip gradients
# Argument for training
n_epochs=41
log_step=200

In [6]:
gino = LitAutoformer(configs)

In [7]:
trainer = L.Trainer(accelerator="gpu",max_epochs=30)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model=gino,train_dataloaders=data, val_dataloaders=vald)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type  | Params
-------------------------------------
0 | autoformer | Model | 171 K 
-------------------------------------
161 K     Trainable params
10.0 K    Non-trainable params
171 K     Total params
0.686     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type  | Params
-------------------------------------
0 | autoformer | Model | 171 K 
-------------------------------------
161 K     Trainable params
10.0 K    Non-trainable params
171 K     Total params
0.686     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [9]:
trainer.validate(model=gino, dataloaders=data)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            kl             │     8.385706901550293     │
│     validation mpjpe      │    106.96685028076172     │
└───────────────────────────┴───────────────────────────┘

[{'validation mpjpe': 106.96685028076172, 'kl': 8.385706901550293}]